In [1]:
import requests
import os
from bs4 import BeautifulSoup

In [2]:
tagPageUrlArray = ["https://classics.autotrader.com/classic-cars-for-sale?seller_type=seller&distance=100",
                   "https://classiccars.com/listings/find?auction=false&dealer=false&private=true",
                   "http://www.rvt.com/New-and-Used-RVs-For-Sale-On-RVT.com/results?private_only=1",
                   "https://www.cars.com/for-sale/searchresults.action/?zc=60606&rd=99999&slrTypeId=28879&yrId=58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199&yrId=20143&yrId=20198&yrId=20142&yrId=20197&yrId=20141&yrId=20180&stkTypId=28881&sf1Nm=price&sf1Dir=DESC&sf2Nm=miles&sf2Dir=ASC&page=1&perPage=50&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&searchSource=GN_BREADCRUMB",
                   "http://www.ksl.com/auto/search/index?keyword=&yearFrom=&yearTo=&mileageFrom=&mileageTo=&priceFrom=25000&priceTo=1000000&zip=&miles=25&newUsed%5B%5D=All&sellerType%5B%5D=For+Sale+By+Owner&postedTime%5B%5D=&titleType%5B%5D=&body%5B%5D=&transmission%5B%5D=&cylinders%5B%5D=&liters%5B%5D=&fuel%5B%5D=&drive%5B%5D=&numberDoors%5B%5D=&exteriorCondition%5B%5D=&interiorCondition%5B%5D=&cx_navSource=hp_search",
                   "http://www.cars-on-line.com/search.html"]
tagPageUrl = tagPageUrlArray[0]

In [3]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

In [4]:
links = tagPageSoup.select('div.pagination > ul > li > a')
max_page = int(links[len(links)-2].get_text())

In [5]:
def has_data_phone_attr(tag):
    return tag.has_attr('data-phone')

In [6]:
for i in range(1, max_page+1):
    tagPageUrl = tagPageUrlArray[0]+"&page="+str(i)
    
    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')
    
    links = tagPageSoup.find_all(has_data_phone_attr)
    for link in links:
        details = link.parent.parent.parent.parent
        name = details.select('a.name')[0]
        model = name.select('h3.model')
        if(len(model)):
            model = model[0].get_text()
        else:
            model = "Unknown"
        price = name.select('h4.price')
        if(len(price)):
            price = price[0].get_text()
        else:
            price = "Unknown"
        phone = link['data-phone']
        bigFile = open('autotrader.txt', 'a')
        bigFile.write(model + '\t' + price + '\t' + phone + '\n')
        bigFile.close()